In [1]:
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

# a method to load a pickle file 

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# load train, test, and meta 

print("Loading Data..")

train_dec = unpickle('./cifar-100-python/train')
test_dec = unpickle('./cifar-100-python/test')
meta = unpickle('./cifar-100-python/meta')

# load data and labels 

train_data = train_dec[b'data']
train_labels = np.array(train_dec[b'fine_labels'])
train_class = np.array(train_dec[b'coarse_labels'])

test_data = test_dec[b'data']
test_labels = np.array(test_dec[b'fine_labels'])
test_class = np.array(test_dec[b'coarse_labels'])

label_names = meta[b'fine_label_names']
class_names = meta[b'coarse_label_names']

Loading Data..


In [2]:
# a method to return dataset in format [num_images,image_height,image_width,num_channels] 
def reshape(dataset):
    return np.reshape(dataset, (-1, 3, 32, 32)).transpose((0, 2, 3, 1))

# a method to shuffle dataset and labels 
def randomize(dataset, labels, classes):
    permutation = np.random.permutation(dataset.shape[0])
    shuffled_dataset = dataset[permutation, :, :]
    shuffled_labels = labels[permutation]
    shuffled_classes = classes[permutation]
    return shuffled_dataset, shuffled_labels, shuffled_classes

# reshape and shuffle 

print("Shuffling Data..")

train_data = reshape(train_data)
test_data = reshape(test_data)
train_data, train_labels, train_class = randomize(train_data, train_labels, train_class)

Shuffling Data..


In [3]:
# a method to sharpen the blurry images 
def sharpen(dataset):
    # img = rgb2gray(img)     
    kernel = (-1 / 256.0) * np.array(
        np.asarray([[1, 4, 6, 4, 1], [4, 16, 24, 16, 4], [6, 24, -476, 24, 6], [4, 16, 24, 16, 4], [1, 4, 6, 4, 1]]))

    for i in range(dataset.shape[0]):
        dataset[i] = cv2.filter2D(dataset[i], -1, kernel)

    return dataset

train_data = sharpen(train_data)
test_data = sharpen(test_data)

In [4]:
pickle_file = 'CIFAR_100_processed.pickle'

# pickle data after pre processing 
try:
    f = open(pickle_file, 'wb')
    save = {
        'train_dataset': train_data,
        'train_labels': train_labels,
        'train_class': train_class,
        'test_dataset': test_data,
        'test_labels': test_labels,
        'test_class': test_class,
        'label_names': label_names,
        'class_names': class_names
    }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    f.close()
    print("Done")
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise

Done
